In [ ]:
def find_lr(model, train_loader, optimizer, criterion, init_value=1e-8, final_value=10.0, beta=0.98):
    num = len(train_loader) - 1
    mult = (final_value / init_value) ** (1/num)
    lr = init_value
    optimizer.param_groups[0]['lr'] = lr
    avg_loss = 0.0
    best_loss = 0.0
    losses = []
    log_lrs = []

    #    for batch_idx, (before, target, future, after, m_b, b_t, m_a, timestamps) in enumerate(train_dataloader):        

    for batch_num, (before, target, future, after, m_b, b_t, m_a, timestamps) in enumerate(train_loader, 1):
        #inputs, targets = data
        optimizer.zero_grad()
        outputs = model(before, after, future)
        loss = criterion(outputs, target)
        avg_loss = beta * avg_loss + (1 - beta) * loss.data.item()
        smoothed_loss = avg_loss / (1 - beta**batch_num)
        if batch_num > 1 and smoothed_loss > 4 * best_loss:
            break
        if smoothed_loss < best_loss or batch_num == 1:
            best_loss = smoothed_loss
        losses.append(smoothed_loss)
        #log_lrs.append(math.log10(lr))
        log_lrs.append(np.log10(lr))

        loss.backward()
        optimizer.step()

        lr *= mult
        optimizer.param_groups[0]['lr'] = lr

    plt.plot(log_lrs[10:-5], losses[10:-5])
    plt.xlabel('Log Learning Rate')
    plt.ylabel('Loss')
    plt.show()

In [ ]:
# TODO: cercare pytorch lighning
train_dataset = CustomDataset(train_df, BATCH_SIZE)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True, drop_last=True)
loss = nn.L1Loss()
model = NeuralNetwork()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

find_lr(model, train_dataloader, optimizer, loss)